In [1]:
# load package
from keras.utils.np_utils import to_categorical
import pandas as pd
import numpy as np
import random
import sys

# fer2013 dataset:
# Training       28709
# PrivateTest     3589
# PublicTest      3589



/usr/share/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# emotion labels from FER2013:
emotion = {'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3,
           'Sad': 4, 'Surprise': 5, 'Neutral': 6}
emo     = ['Angry', 'Fear', 'Happy',
           'Sad', 'Surprise', 'Neutral']

In [3]:
#load data
filepath = 'fer2013.csv'
df = pd.read_csv(filepath)

In [4]:
#explore data
print (df.shape)
print (df.tail())
print (df.Usage.value_counts())
#split the train and test from the whole dataset.
df_train = df[df.Usage == 'Training']
print (df_train.shape)
df_test = df[df.Usage == 'PublicTest']
print (df_test.shape)

(35887, 3)
       emotion                                             pixels        Usage
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest
Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64
(28709, 3)
(3589, 3)


In [5]:
#split the sample
sample_split=0.02
frames = []
classes=['Angry', 'Fear', 'Happy',
           'Sad', 'Surprise', 'Neutral']

for _class in classes:
    class_df = df[df['emotion'] == emotion[_class]]
    frames.append(class_df)
data = pd.concat(frames, axis=0)
print (data.shape)
print (data.tail())
print (data.Usage.value_counts())
print (data.emotion.value_counts())

rows = random.sample(list(data.index), int(len(data)*sample_split))
print (len(rows))
data1 = data.loc[rows]
print (data1.tail())
print (data1.emotion.value_counts())
print (data1.Usage.value_counts())

(35340, 3)
       emotion                                             pixels        Usage
35866        6  87 82 59 61 72 102 143 130 90 95 143 173 146 1...  PrivateTest
35871        6  112 102 98 89 98 133 164 185 180 179 185 169 1...  PrivateTest
35876        6  29 29 27 31 49 56 29 19 22 20 34 43 55 71 85 9...  PrivateTest
35877        6  139 143 145 154 159 168 176 181 190 191 195 19...  PrivateTest
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
Training       28273
PrivateTest     3534
PublicTest      3533
Name: Usage, dtype: int64
3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
Name: emotion, dtype: int64
706
       emotion                                             pixels        Usage
23631        2  255 251 149 114 113 101 112 112 119 134 143 14...     Training
34247        4  95 37 5 0 26 141 180 185 183 179 168 128 96 95...  PrivateTest
20517        6  214 215 236 251 255 255 255 255 254 255 200 14...     Training
102          6  58

In [34]:
#reconstruct data pixels, from string to int

data_train = data1[data1.Usage == 'Training']
data_test = data1[data1.Usage == 'PublicTest']

#reconstruct train x and y
data_train_x = list(data_train["pixels"])
data_train_y = list(data_train["emotion"])

train_x = []
for i in range(len(data_train_x)):
    each_pixel = [int(num) for num in data_train_x[i].split()]
    train_x.append(each_pixel)
train_x = np.array(train_x)
train_y = np.array(data_train_y )


print (train_x.shape)
print (train_y.shape)


#reconstruct test x and y
data_test_x = list(data_test["pixels"])
data_test_y = list(data_test["emotion"])

test_x = []
for i in range(len(data_test_x)):
    each_pixel = [int(num) for num in data_test_x[i].split()]
    test_x.append(each_pixel)
test_x = np.array(test_x)
test_y = np.array(data_test_y )


print (test_x.shape)
print (test_y.shape)




# to categorical

print (train_y.shape)

(573, 2304)
(573,)
(61, 2304)
(61,)
(573,)


In [35]:
#train CNN
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

batch_size = 128
nb_classes = 7
nb_epoch = 16

# input image dimensions
img_rows, img_cols = 48, 48

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

X_train = train_x
X_test = test_x
Y_train = train_y
Y_test = test_y

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols,1)
X_test = X_test.reshape(X_test.shape[0],  img_rows, img_cols,1)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

print (Y_train.shape)

X_train shape: (573, 48, 48, 1)
573 train samples
61 test samples
(573, 7)


In [39]:
from keras import layers
from keras import models
from keras import optimizers

model = Sequential()
model.add(layers.Conv2D(nb_filters, (img_rows, img_cols), activation='relu',
                        input_shape=( img_rows, img_cols,1)))


model.add(layers.Flatten())


model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))



model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 1, 1, 32)          73760     
_________________________________________________________________
flatten_11 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               16896     
_________________________________________________________________
dense_14 (Dense)             (None, 7)                 3591      
Total params: 94,247
Trainable params: 94,247
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch)
#score = model.evaluate(X_test, Y_test)
#print('Test score:', score[0])
#print('Test accuracy:', score[1])

/usr/share/anaconda3/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/16
573/573 [==============================] - 1s 943us/step - loss: 1.9172 - acc: 0.2373
Epoch 2/16
573/573 [==============================] - 0s 49us/step - loss: 1.8716 - acc: 0.2496
Epoch 3/16
573/573 [==============================] - 0s 51us/step - loss: 1.8461 - acc: 0.2496
Epoch 4/16
573/573 [==============================] - 0s 49us/step - loss: 1.8311 - acc: 0.2496
Epoch 5/16
573/573 [==============================] - 0s 49us/step - loss: 1.8192 - acc: 0.2496
Epoch 6/16
573/573 [==============================] - 0s 46us/step - loss: 1.8072 - acc: 0.2496
Epoch 7/16
573/573 [==============================] - 0s 49us/step - loss: 1.8005 - acc: 0.2496
Epoch 8/16
573/573 [==============================] - 0s 59us/step - loss: 1.7917 - acc: 0.2496
Epoch 9/16
573/573 [==============================] - 0s 51us/step - loss: 1.7869 - acc: 0.2496
Epoch 10/16
573/573 [==============================] - 0s 49us/step - loss: 1.7803 - acc: 0.2496
Epoch 11/16
573/573 [=================

In [ ]:
#then try RNN